In [1]:
import pandas as pd
import os
import re
import tqdm
from typing import List, Dict
import json

In [ ]:
# swapped_bruk_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped/bruk_changed"
# swapped_ng_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped/ng_changed"

# orig_bruk_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_orig/bruk"
# orig_ng_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_orig/ng"

swapped_bruk_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk_changed"
swapped_ng_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed"

orig_bruk_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/bruk"
orig_ng_folder = "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng"

## Creating parallel dataset

In [ ]:
def parse_ann_file(ann_file):
    entities = []

    with open(ann_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                parts = line.strip().split("\t")
                if len(parts) < 3:
                    continue
                label = parts[1]
                start = parts[2]
                end = parts[3]
                text = parts[4]

                entities.append({"ind": parts[0], "text": text, "start": start, "end": end, "label": label})
            except:
                continue
                # print("issue with row:", line)

    return entities


def extract_entities(txt_file: str, ann_file: str):
    labeled_sentences_list = []
    sentences = []

    with open(txt_file, "r", encoding="utf-8") as f:
        sentence_lines = f.readlines()

    entities = parse_ann_file(ann_file)
    empty_lines_count = sentence_lines.count("\n")
    
    actual_counter = -1
    used_tags = []
    sent_end = 0
    for ind, sentence in enumerate(sentence_lines):
        actual_counter += 1
        sent_end += len(sentence)
        labeled_dict = {}
        job = False

        if sentence == "\n":
            actual_counter -= 1
        sentence = sentence.strip()

        for ent in entities:
            if re.search(rf"\b{re.escape(ent['text'])}\b", sentence):
                if ent["label"] == "JOB" and ent['ind'] not in used_tags and int(ent['start']) < sent_end:
                    labeled_dict[ent['ind']] = (ent["text"], ent["label"])
                    used_tags.append(ent['ind'])
                    job = True

        if job:  # Save sentences only if they contain a JOB entity
            labeled_sentences_list.append(labeled_dict)
            sentences.append((ind, sentence, actual_counter))

    return labeled_sentences_list, sentences, empty_lines_count, len(sentence_lines)


def get_sentences_with_job_titles(txt_file_names: List[str]):
    sentence_with_jobs = {}

    for txt_file in txt_file_names:
        ann_file = txt_file.replace(".txt", ".ann")
        labeled_sentences_list, sentences, empty_lines_count, numb_of_sents = extract_entities(txt_file, ann_file)
        for i in range(len(sentences)):
            sentence_with_jobs[sentences[i][1]] = (sentences[i][0], labeled_sentences_list[i],  empty_lines_count, numb_of_sents, sentences[i][2])

    return sentence_with_jobs


def process_files(original_folder: str, changed_folder: str):
    dataset = []

    original_files = [f for f in os.listdir(original_folder) if f.endswith('.txt')]
    # changed_files = [f for f in os.listdir(changed_folder) if f.endswith('.txt')]

    for filename in tqdm.tqdm(original_files):
        # try:
        orig_txt_path = os.path.join(original_folder, filename)
        changed_txt_path = os.path.join(changed_folder, filename.replace('.txt', '_1.txt'))

        if not os.path.exists(changed_txt_path):
            raise FileNotFoundError(f"Missing corresponding file for {filename} in the changed folder.")

        job_sentences = get_sentences_with_job_titles([orig_txt_path])

        with open(changed_txt_path, 'r', encoding='utf-8') as f:
            changed_sentences = f.readlines()

        for orig_sent, val in job_sentences.items():
            # val[0]
            if len(changed_sentences) != val[3]:
                if len(changed_sentences) + val[2] == val[3]:
                    # print(len(val))
                    # print(val[4])
                    # if len(val) <5:
                    #     print("here")
                    #     print(val)
                    #     # return
                    changed_sent = changed_sentences[val[4]]
                    dataset.append([orig_sent, val[0], changed_sent, val[4], filename, val[1]])
                else:
                    print(f"Cannot process file {filename}")
                    print("need len:", val[3], 'empty sents:', val[2])
                    print("actual len:", len(changed_sentences))
                    return
            else:
                changed_sent = changed_sentences[val[0]]
                dataset.append([orig_sent, val[0], changed_sent.strip(), val[0], filename, val[1]])
            # except Exception as e:
            #     print(f"Cannot process file totaly {filename}: {str(e)} \n val: {val}")
        # for orig_sentence in job_sentences.keys():
        #     if orig_sentence in changed_sentences:
        #         dataset.append([orig_sentence, orig_sentence, filename])
    # except Exception as e:
    #     print(f"Error processing file {filename}: {str(e)}")

    columns = ["original_sentence", "orig_sent_id", "changed_sentence", "changed_sent_id", "original_file_name", "orig_ann"]
    return pd.DataFrame(dataset, columns=columns)

In [ ]:
# with open("/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_orig/bruk/c0c436cc9cb6.txt", 'r', encoding='utf-8') as f:
#     orig_sent = f.readlines()
#     print(len(orig_sent))

44


In [49]:
parallel_dataset_bruk_dataset = process_files(orig_bruk_folder, swapped_bruk_folder)

100%|██████████| 262/262 [00:06<00:00, 40.34it/s]


In [ ]:
# parallel_dataset_bruk_dataset[parallel_dataset_bruk_dataset['original_file_name']=='0ac5140eb732.txt']

In [52]:
parallel_dataset_bruk_dataset.head()

,original_sentence,orig_sent_id,changed_sentence,changed_sent_id,original_file_name,orig_ann
0,Його редактором був поет-символіст Яків Савчен...,46,Його редакторкою була поетеса-символістка Анже...,46,e5e76a8efa0f.txt,"{'T30': ('поет-символіст', 'JOB')}"
1,Міжпредметні паралелі . Маніфест футуристів ск...,62,Міжпредметні паралелі . Маніфест футуристок ск...,62,e5e76a8efa0f.txt,"{'T49': ('поет', 'JOB')}"
2,Помітною була організація « Гарт » ( 1923 — 19...,96,Помітною була організація « Гарт » ( 1923 — 19...,96,e5e76a8efa0f.txt,"{'T104': ('поет', 'JOB')}"
3,Її очолив байкар і прозаїк Сергій Пилипенко .,127,Її очолила байкарка і прозаїкиня Марія Пилипен...,127,e5e76a8efa0f.txt,"{'T143': ('байкар', 'JOB'), 'T144': ('прозаїк'..."
4,У Галицько-Волинському літописі згадується спі...,171,У Галицько-Волинському літописі згадується спі...,171,e5e76a8efa0f.txt,"{'T218': ('співець', 'JOB')}"


In [53]:
parallel_dataset_ng_dataset = process_files(orig_ng_folder, swapped_ng_folder)

  0%|          | 0/298 [00:00<?, ?it/s]

100%|██████████| 298/298 [00:03<00:00, 90.87it/s] 


In [56]:
parallel_dataset_ng_dataset.head()

,original_sentence,orig_sent_id,changed_sentence,changed_sent_id,original_file_name,orig_ann,changed_ann
0,"Слідчі встановили, що у 2018-2019 рр. управлін...",2,"Слідчі встановили, що у 2018-2019 рр. управлін...",2,104263660695.txt,"{'T6': ('Слідчі', 'JOB')}","{'T6': ('Слідчі', 'JOB')}"
1,Суд надав слідчим доступ до документації щодо ...,7,Суд надав слідчій доступ до документації щодо ...,7,104263660695.txt,"{'T23': ('слідчим', 'JOB')}","{'T23': ('слідчій', 'JOB')}"
2,"Директором та засновником ТОВ «Армшляхбуд», ст...",8,"Директоркою та засновницею ТОВ «Армшляхбуд», с...",8,104263660695.txt,"{'T25': ('Директором', 'JOB'), 'T26': ('заснов...","{'T25': ('Директоркою', 'JOB'), 'T26': ('засно..."
3,Співвласником цієї компанії у свою чергу з час...,5,Співвласницею цієї компанії у свою чергу з час...,5,97c7987b8f9c.txt,"{'T17': ('голова наглядової ради', 'JOB')}","{'T17': ('голова наглядової ради', 'JOB')}"
4,Одесит Іван Плачков почав працювати в компанії...,6,Одеситка Іванна Плачкова почала працювати в ко...,6,97c7987b8f9c.txt,"{'T25': ('головою правління', 'JOB')}","{'T25': ('головою правління', 'JOB')}"


In [63]:
parallel_dataset_ng_dataset[parallel_dataset_ng_dataset['original_file_name']=='0e31947e4561.txt']


,original_sentence,orig_sent_id,changed_sentence,changed_sent_id,original_file_name,orig_ann,changed_ann
620,Службові особи лісових господарств Житомирсько...,0,Службові особи лісових господарств Житомирсько...,0,0e31947e4561.txt,"{'T2': ('підрядниками', 'JOB')}","{'T2': ('підрядницями', 'JOB')}"
621,"Приміром, слідство вказує, що одним із підрядн...",4,"Приміром, слідство вказує, що одним із підрядн...",4,0e31947e4561.txt,"{'T10': ('підрядників', 'JOB')}","{'T10': ('підрядниць', 'JOB')}"
622,"Слідчі встановили, що значну частину робіт для...",5,"Слідчі встановили, що значну частину робіт для...",5,0e31947e4561.txt,"{'T12': ('Слідчі', 'JOB')}","{'T12': ('Слідчі', 'JOB')}"
623,Суд надав слідчим доступ до документації остан...,7,Суд надав слідчим доступ до документації остан...,7,0e31947e4561.txt,"{'T15': ('слідчим', 'JOB')}","{'T15': ('слідчим', 'JOB')}"
624,У 2011 році директором фірми був тодішній депу...,9,У 2011 році директоркою фірми була тодішня деп...,9,0e31947e4561.txt,"{'T21': ('директором', 'JOB'), 'T22': ('депута...","{'T21': ('директоркою', 'JOB'), 'T22': ('депут..."
625,Надалі засновниками компанії ставали лондонськ...,12,Надалі засновницями компанії ставали лондонськ...,12,0e31947e4561.txt,"{'T30': ('засновниками', 'JOB'), 'T32': ('засн...","{'T30': ('засновницями', 'JOB'), 'T32': ('засн..."
626,"Нинішнім засновником і директором компанії, як...",13,"Нинішньою засновницею і директоркою компанії, ...",13,0e31947e4561.txt,"{'T38': ('засновником', 'JOB'), 'T39': ('дирек...","{'T38': ('засновницею', 'JOB'), 'T39': ('дирек..."
627,Засновником і директором ПП «Індустріальна ком...,15,Засновницею і директоркою ПП «Індустріальна ко...,15,0e31947e4561.txt,"{'T48': ('Засновником', 'JOB'), 'T49': ('дирек...","{'T48': ('Засновницею', 'JOB'), 'T49': ('дирек..."


In [60]:
def find_changed_ann(orig_annotation, filename, path):
    
    list_of_inds = [ind for ind, ent in orig_annotation.items()]

    changed_ann_file = path + filename.replace(".txt", "_1.ann")

    entities = parse_ann_file(changed_ann_file)
    labeled_dict = {}

    for ent in entities:
           if ent['ind'] in list_of_inds:
                labeled_dict[ent["ind"]] = (ent['text'], ent["label"])
    return labeled_dict

parallel_dataset_bruk_dataset['changed_ann'] = parallel_dataset_bruk_dataset.apply(lambda x: find_changed_ann(x['orig_ann'], x['original_file_name'], "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped/bruk_changed/"), axis=1)
parallel_dataset_ng_dataset['changed_ann'] = parallel_dataset_ng_dataset.apply(lambda x: find_changed_ann(x['orig_ann'], x['original_file_name'], "/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_swapped/ng_changed/"), axis=1)

In [ ]:
# /Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_orig/bruk/87ae82667e2a.txt 

In [61]:
parallel_dataset_bruk_dataset.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/bruk_parallel.csv", index=0)
parallel_dataset_ng_dataset.to_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv", index=0)

In [62]:
parallel_dataset_bruk_dataset[parallel_dataset_bruk_dataset['orig_sent_id'] != parallel_dataset_bruk_dataset['changed_sent_id']]

,original_sentence,orig_sent_id,changed_sentence,changed_sent_id,original_file_name,orig_ann,changed_ann
41,В наших умовах у таких « рольових коконах » пр...,6,В наших умовах у таких « рольових коконах » пр...,3,87ae82667e2a.txt,"{'T3': ('лектори', 'JOB')}","{'T3': ('лекторки', 'JOB')}"
42,"Можливо , комусь ця думка здасться блюзнірчою ...",16,"Можливо , комусь ця думка здасться блюзнірчою ...",8,87ae82667e2a.txt,"{'T18': ('режисером', 'JOB')}","{'T18': ('режисеркою', 'JOB')}"
73,Коломийська влада в особі міського голови Ігор...,2,Коломийська влада в особі міської голови Анаст...,1,e010e86e436c.txt,"{'T5': ('міського голови', 'JOB'), 'T7': ('нач...","{'T5': ('міської голови', 'JOB'), 'T7': ('нача..."
74,"Народний депутат України Андрій Іванчук , пере...",7,"Народна депутатка України Марія Іванчук , пере...",5,e010e86e436c.txt,"{'T10': ('Народний депутат', 'JOB')}","{'T10': ('Народна депутатка', 'JOB')}"
75,Львів ще від часів українських королів і князі...,23,Львів ще від часів українських королев і княги...,12,7b2c9399fee9.txt,"{'T26': ('королів', 'JOB'), 'T27': ('князів', ...","{'T26': ('королев', 'JOB'), 'T27': ('княгинь',..."
...,...,...,...,...,...,...,...
259,"І Лянґе зрозумів , що він більше не має сил об...",25,"І Лянґе зрозуміла , що вона більше не має сил ...",13,ef1b6bd52747.txt,"{'T31': ('мовознавцем', 'JOB')}","{'T31': ('мовознавицею', 'JOB')}"
293,Тож фон Трієру я винна окрему дяку : з усіх мо...,4,Тож фон Трієру я винна окрему дяку : з усіх мо...,2,f5bb37569413.txt,"{'T2': ('режисера', 'JOB')}","{'T2': ('режисерки', 'JOB')}"
294,"З погляду бджоли , різниці між мною і колишнім...",48,"З погляду бджоли , різниці між мною і колишньо...",24,f5bb37569413.txt,"{'T24': ('наркомом', 'JOB'), 'T25': ('головою'...","{'T24': ('наркомкою', 'JOB'), 'T25': ('головою..."
295,І за яким хріном ми з академіком Петросянцем б...,56,І за яким хріном ми з академікинею Петросянець...,28,f5bb37569413.txt,"{'T30': ('академіком', 'JOB')}","{'T30': ('академікинею', 'JOB')}"


In [7]:
parallel_dataset_ng_dataset = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/csv_files_with_par_sentences/ng_parallel.csv")

## Split: Female, Male, Common genders

In [ ]:
with open('/Users/linndfors/study/diploma/uk-gender-word-mapper/common_gender_words_list.txt') as file:
    common_gender_words_list = [line.strip() for line in file if line.strip()]

with open('/Users/linndfors/study/diploma/uk-gender-word-mapper/male_words_list.txt') as file:
    male_list = [line.strip() for line in file if line.strip()]

with open('/Users/linndfors/study/diploma/uk-gender-word-mapper/female_words_list.txt') as file:
    female_list = [line.strip() for line in file if line.strip()]

gender_dict_df = pd.read_csv("/Users/linndfors/study/diploma/uk-gender-word-mapper/gender_pairs_dictionary.csv")

gender_dict = {'male': [], 'female': []}

for _, row in gender_dict_df.iterrows():
    gender_dict['male'].append(row['male'])
    female_values = [f.strip() for f in row['female'].split(',')]
    gender_dict['female'].extend(female_values)

gender_dict['female'] = list(set(gender_dict['female']))

exceptions_male = ['прокурор', 'віце-премʼєра', 'премʼєр', 'начальника', 'начальником', 'міністра', 'директором', 'підрядником', 'аграрія', 'депутата', 'генпідрядника', 'начальник', 'керівником', 'керівник']
exceptions_female = ['слідчій', 'прокурорка', 'прокурорки', 'Прокурорки', 'прокуроркою', 'прокурорки', 'адміністраторок', 'прокурорці', 'слідчу', 'слідча' , 'начальницею', 'податківка', 'монопольниці' "начальницею", "начальницю", "психіатрині", 'наркологині', 'міністерки', 'рітейлерками', 'начальниці', 'екологині', 'податківчині', 'журналісткам', 'начальниці', 
'підрядницею', 'директорка', 'інженерка', 'начальниця', 'службовиці', 'інженерці', 'податківниць', 'депутатку', 'керівниця', 'начальницею'
'керівницею', 'юристок', 'інспекторки', 'інженерок', 'ревізорок', 'нардепок', 'бізнесменок', 'начальницю', 'генпідрядниці', ]

In [ ]:
import stanza
import pymorphy3
import pymorphy2

nlp = stanza.Pipeline('uk', processors='tokenize,mwt,pos,lemma,depparse')
morph = pymorphy3.MorphAnalyzer()


# # stanza.download('uk')
# nlp = stanza.Pipeline('uk', processors='tokenize,mwt,pos,lemma')

In [ ]:
# def extract_main_word(text):
#     # Process the text with Stanza
#     doc = nlp(text)
    
#     # Check the dependency parse for the main word
#     for sentence in doc.sentences:
#         # Get the head word based on dependencies
#         for word in sentence.words:
#             if word.head == 0:  # The head of the sentence (the main word)
#                 return word.lemma

#     # Fallback if no head word found
#     return None

# # Test the function
# text = "докторантка відділу соціальної антропології"
# main_word = extract_main_word(text)
# print(main_word)

докторантка


In [ ]:
# entity = 'судді'
# job = nlp(entity)

# job_ent_list = [word.lemma for sent in job.sentences for word in sent.words]
# job_lemma = " ".join(job_ent_list)

# print("Corrected lemma:", job_lemma)

# # if len(job_ent_list) > 1:
# #     job_lemma = extract_main_word(job_lemma)
# #     print("main word:", job_lemma)

lemma: судді
Corrected lemma: судді


In [ ]:
# text = "топ-менеджерів"
# main_word = extract_main_word(text)
# print(f"Main word of '{text}': {main_word}")

In [ ]:

# # Initialize pymorphy3 analyzer

# def get_lemma_from_pymorphy(entity):
#     # Split the entity into words and lemmatize each word
#     words = entity.split()  # Split by spaces to handle multi-word entities
#     lemmatized_words = [morph.parse(word)[0].normal_form for word in words]
    
#     # Join the lemmatized words back into a single string
#     return " ".join(lemmatized_words)
# get_lemma_from_pymorphy("голові")


In [71]:
def extract_main_word(text):
    if "-" in text:
        parts = text.split("-")
        main_word = parts[-1]
        doc = nlp(main_word)
        for sentence in doc.sentences:
            for word in sentence.words:
                return word.lemma

    doc = nlp(text)


    for sentence in doc.sentences:
        for word in sentence.words:
            if word.head == 0:
                main_word = word
                if main_word.text in {"рок", "анти", "псевдо", "віце", "топ"}:
                    continue
                return main_word.lemma
    for sentence in doc.sentences:
        for word in sentence.words:
            return word.lemma
        
def extract_gender(entity):
    entity = entity.lower()
    job = nlp(entity)
    join_sign = " " if " " in entity else ("" if "-" in entity else "")
    job_ent_list = [word.lemma for sent in job.sentences for word in sent.words]
    job_lemma = join_sign.join(job_ent_list)

    if len(job_ent_list) > 1:
        job_lemma = extract_main_word(job_lemma)

    words = entity.split()
    lemmatized_words = []
    for word in words:
        parsed_word = morph.parse(word)[0]
        if 'plur' in parsed_word.tag:
            singular_form = parsed_word.inflect({'sing'}).word if parsed_word.inflect({'sing'}) else parsed_word.normal_form
            lemmatized_words.append(singular_form)
        else:
            lemmatized_words.append(parsed_word.normal_form)
            
    job_lemma_pymorphy = join_sign.join(lemmatized_words)

    if len(job_ent_list) > 1:
        job_lemma_pymorphy = extract_main_word(job_lemma_pymorphy)
        
    for x in [entity, job_lemma, job_lemma_pymorphy]:
        for common_word in ["судді", "ченці", "ченців", "глави", "голови", "керівництво", "в. о.", "головою"]:
            if common_word in x:
                return "common"
            
        for female_word in exceptions_female:
            if female_word in x:
                return "female"
            
        for male_word in exceptions_male:
            if male_word in x:
                return "male"
            
        if (x in gender_dict['female']) or (x in female_list) or ("знавиця" in x):
            return "female"
        elif (x in gender_dict['male']) or (x in male_list) or ("знавець" in x) or (x=="мера"):
            return "male"
        elif x in common_gender_words_list:
            return "common"
        
    print("unkown for:", entity, " - ", job_lemma, " - ", job_lemma_pymorphy)
    return "unknown_gender"

### Original entitites

In [72]:
import ast

file_gender_dict = {"male": {}, "female": {}, "common": {}, "unknown_gender": {}}

total_job_counter = 0
job_list = []

for x, row in tqdm.tqdm(parallel_dataset_ng_dataset.iterrows()):
    filename = row['original_file_name']
    orig_annotation = row['orig_ann']

    ann_str = orig_annotation.replace("'", '"')
    ann_str = ann_str.replace("–", '-')
    json_ann = ast.literal_eval(ann_str)
    
    try:
        for ent, feat in json_ann.items():
            if feat[1] == 'JOB':
                total_job_counter += 1
                job_list.append(feat[0])
                gender_value = extract_gender(feat[0])
                
                if filename not in file_gender_dict[gender_value]:
                    file_gender_dict[gender_value][filename] = [(ent, feat[0])]
                else:
                    file_gender_dict[gender_value][filename].append((ent, feat[0]))
                
    except Exception as e:
        print(f"Issue with row: {row} - Error: {e}")

178it [01:01,  5.16it/s]

unkown for: податківці  -  податок  -  податківці


506it [02:34,  4.21it/s]

unkown for: головні мисливствознавці держлісгоспів  -  мисливствітнавка  -  мисливствітнавка


563it [02:47,  4.98it/s]

...

687it [03:29,  2.89it/s]

unkown for: директор департаменту з питань традиційних та альтернативних палив  -  департамент  -  палити


865it [04:45,  3.87it/s]

unkown for: податківців  -  податокець  -  податківціть


1021it [05:36,  3.04it/s]


In [73]:
print("total size:", total_job_counter)

for gender_class, val in file_gender_dict.items():
    print("gender:", gender_class)
    print("size:", len(file_gender_dict[gender_class]))

total size: 1338
gender: male
size: 269
gender: female
size: 16
gender: common
size: 93
gender: unknown_gender
size: 6


In [80]:
orig_female_files = []
orig_male_files = []
orig_common_files = []

In [81]:
for gen, file_with_ent in file_gender_dict.items():
    for filename, ent_with_ind in file_with_ent.items():
        if gen == "male":
            orig_male_files.append(filename)
        elif gen == "female":
            orig_female_files.append(filename)
        elif gen == "common":
            orig_common_files.append(filename)

### Swapped entitites

In [76]:
file_gender_dict_swapped = {"male": {}, "female": {}, "common": {}, "unknown_gender": {}}

total_job_counter_swapped = 0
job_list_swapped = []

for x, row in tqdm.tqdm(parallel_dataset_ng_dataset.iterrows()):
    filename = row['original_file_name']
    filename = filename.replace(".txt", "_1.txt")
    changed_annotation = row['changed_ann']

    ann_str = changed_annotation.replace("'", '"')
    ann_str = ann_str.replace("–", '-')
    json_ann = ast.literal_eval(ann_str)
    
    try:
        for ent, feat in json_ann.items():
            if feat[1] == 'JOB':
                total_job_counter_swapped += 1
                job_list_swapped.append(feat[0])
                gender_value = extract_gender(feat[0])

                if (feat[0]=='слідчій'):
                    print("gender:", gender_value)
                
                if filename not in file_gender_dict_swapped[gender_value]:
                    file_gender_dict_swapped[gender_value][filename] = [(ent, feat[0])]
                else:
                    file_gender_dict_swapped[gender_value][filename].append((ent, feat[0]))
                
    except Exception as e:
        print(f"Issue with row: {row} - Error: {e}")

2it [00:00,  8.22it/s]

gender: female


44it [00:15,  7.17it/s]

gender: female


84it [00:22,  6.54it/s]

...

746it [03:21,  2.31it/s]

unkown for: сбушниці  -  сбушниця  -  сбушниці


962it [04:33,  1.28it/s]

unkown for: світлана зубачик  -  зубачик  -  зубачик


1021it [04:51,  3.50it/s]


In [77]:
print("total size:", total_job_counter_swapped)

for gender_class, val in file_gender_dict_swapped.items():
    print("gender:", gender_class)
    print("size:", len(file_gender_dict_swapped[gender_class]))

total size: 1338
gender: male
size: 107
gender: female
size: 260
gender: common
size: 91
gender: unknown_gender
size: 14


In [82]:
swapped_female_files = []
swapped_male_files = []
swapped_common_files = []

In [83]:
for gen, file_with_ent in file_gender_dict_swapped.items():
    for filename, ent_with_ind in file_with_ent.items():
        if gen == "male":
            swapped_male_files.append(filename)
        elif gen == "female":
            swapped_female_files.append(filename)
        elif gen == "common":
            swapped_common_files.append(filename)

In [84]:
# swapped_female_files_global = ["/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/bruk_changed/" + filename for filename in swapped_female_files]
# swapped_male_files_global = ["/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/bruk_changed/" + filename for filename in swapped_male_files]
# swapped_common_files_global = ["/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/bruk_changed/" + filename for filename in swapped_common_files]

swapped_female_files_global = ["/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed/" + filename for filename in swapped_female_files]
swapped_male_files_global = ["/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed/" + filename for filename in swapped_male_files]
swapped_common_files_global = ["/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng_changed/" + filename for filename in swapped_common_files]

In [85]:
# orig_male_files = ["/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/bruk/" + filename for filename in orig_male_files]
# orig_female_files = ["/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/bruk/" + filename for filename in orig_female_files]
# orig_common_files = ["/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_balanced/bruk/" + filename for filename in orig_common_files]

orig_male_files = ["/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/" + filename for filename in orig_male_files]
orig_female_files = ["/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/" + filename for filename in orig_female_files]
orig_common_files = ["/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER/ng/" + filename for filename in orig_common_files]

In [86]:
# bruk_male_files = swapped_male_files_global + orig_male_files
# bruk_female_files = swapped_female_files_global + orig_female_files
# bruk_common_files = swapped_common_files_global + orig_common_files

ng_male_files = swapped_male_files_global + orig_male_files
ng_female_files = swapped_female_files_global + orig_female_files
ng_common_files = swapped_common_files_global + orig_common_files

In [88]:
import shutil

Push files with specific gender to their directories

In [ ]:
# # TODO

# find this file in "/Users/linndfors/study/diploma/ner_for_fem/data/DATA_FOR_BALANCED_NER"
# if it is swapped check if I push right annotation (only pers and job)

In [100]:
# for male_file in ng_male_files:
#     # source_path = male_file
#     source_path = male_file.replace(".txt", ".ann")
#     destination_dir = '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_male/ng'

#     file_name = os.path.basename(source_path)

#     destination_path = os.path.join(destination_dir, file_name)

#     shutil.copy(source_path, destination_path)

# for female_file in ng_female_files:
#     # source_path = female_file
#     source_path = female_file.replace(".txt", ".ann")
#     destination_dir = '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_female/ng'

#     file_name = os.path.basename(source_path)

#     destination_path = os.path.join(destination_dir, file_name)

#     shutil.copy(source_path, destination_path)

for common_file in ng_common_files:
    # source_path = common_file
    source_path = common_file.replace(".txt", ".ann")
    destination_dir = '/Users/linndfors/study/diploma/ner_for_fem/data/data_for_ner/data_for_ner_gender/data_for_ner_common/ng'

    file_name = os.path.basename(source_path)

    destination_path = os.path.join(destination_dir, file_name)

    shutil.copy(source_path, destination_path)